In [58]:
# multi-class classification with Keras
import pandas
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [84]:
df = pandas.read_csv("CI_labelled.csv")
df

,RMS,Crest_factor,Kurtosis,Standard_Deviation,Label
0,0.187225,4.886918,4.024185,0.187240,C1
1,0.171226,5.693860,4.409202,0.171239,C1
2,0.189137,5.643351,5.077067,0.189152,C1
3,0.189435,6.646050,5.345039,0.189449,C1
4,0.202397,7.563921,6.365882,0.202412,C1
...,...,...,...,...,...
7995,0.030333,10.860104,7.308769,0.030336,C0
7996,0.041194,12.189303,16.107614,0.041197,C0
7997,0.049760,14.597865,32.637704,0.049764,C0
7998,0.062941,17.900079,35.452820,0.062946,C0


In [132]:
win_len=100
stride=10
X=[]
Y=[]
for i in range(0,4):
    df_temp = df[df['Label']=='C{}'.format(i)]
    for j in np.arange(0,len(df_temp)-(win_len),stride):
        X.append(df_temp.iloc[i:i+win_len,:-1])
        Y.append(df_temp.iloc[i+win_len,-1])
    
X=np.array(X)
X=X.reshape((X.shape[0],X.shape[1],X.shape[2],1))
Y=np.array(Y)

In [133]:
X.shape

(760, 100, 4, 1)

In [134]:
Y.shape

(760,)

In [135]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [136]:
dummy_y

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32)

In [137]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, dummy_y, test_size=0.33,shuffle=True)

In [151]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten
# create model
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size=(20, 3),activation='relu',input_shape=(X.shape[1],X.shape[2],1),padding='same'))
cnn_model.add(MaxPooling2D((20, 2),strides=(5, 5),padding='same'))
cnn_model.add(Conv2D(64, (10, 3), activation='relu',padding='same'))
cnn_model.add(MaxPooling2D(pool_size=(10, 2),strides=(3, 3),padding='same'))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dense(4, activation='softmax'))
cnn_model.summary()
cnn_model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

Model: "sequential_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_23 (Conv2D)          (None, 100, 4, 32)        1952      
                                                                 
 max_pooling2d_23 (MaxPoolin  (None, 20, 1, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_24 (Conv2D)          (None, 20, 1, 64)         61504     
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 7, 1, 64)         0         
 g2D)                                                            
                                                                 
 flatten_12 (Flatten)        (None, 448)               0         
                                                                 
 dense_62 (Dense)            (None, 128)             

In [153]:
epochs = 5
batch_size=128
history = cnn_model.fit(X_train, Y_train, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(X_test,Y_test),shuffle=True)

Epoch 1/5
4/4 [==============================] - 0s 43ms/step - loss: 0.5630 - accuracy: 0.7544 - val_loss: 0.4895 - val_accuracy: 0.7410
Epoch 2/5
4/4 [==============================] - 0s 33ms/step - loss: 0.4128 - accuracy: 0.9332 - val_loss: 0.3540 - val_accuracy: 1.0000
Epoch 3/5
4/4 [==============================] - 0s 33ms/step - loss: 0.3007 - accuracy: 1.0000 - val_loss: 0.2360 - val_accuracy: 1.0000
Epoch 4/5
4/4 [==============================] - 0s 33ms/step - loss: 0.2009 - accuracy: 1.0000 - val_loss: 0.1567 - val_accuracy: 1.0000
Epoch 5/5
4/4 [==============================] - 0s 32ms/step - loss: 0.1268 - accuracy: 1.0000 - val_loss: 0.0907 - val_accuracy: 1.0000


In [158]:
y_pred=cnn_model.predict(X_test)
y_pred

8/8 [==============================] - 0s 4ms/step


array([[3.1761345e-03, 7.9342583e-04, 6.4797525e-05, 9.9596560e-01],
       [3.1761345e-03, 7.9342583e-04, 6.4797525e-05, 9.9596560e-01],
       [2.9195590e-02, 5.0529964e-02, 9.1647220e-01, 3.8021633e-03],
       ...,
       [9.3103069e-01, 4.2658668e-02, 1.9109691e-02, 7.2009130e-03],
       [9.3103069e-01, 4.2658668e-02, 1.9109691e-02, 7.2009130e-03],
       [9.3103069e-01, 4.2658668e-02, 1.9109691e-02, 7.2009130e-03]],
      dtype=float32)